In [1]:
!pip3 install opencv-python
!pip3 install ultralytics
!pip3 install tensorflow
!pip3 install dill
!clear

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 671.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.4/869.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.3 MB/s eta 0:00:00


In [2]:
import os
from google.colab import drive

def mount_drive_if_not_mounted(mount_point='/content/drive'):
    """
    Checks if Google Drive is mounted at the specified mount_point.
    If not, mounts the drive.

    Parameters:
    - mount_point (str): The directory where Google Drive is mounted.
    """
    # Define the path to 'My Drive' within the mount point
    my_drive_path = os.path.join(mount_point, 'My Drive')

    # Check if 'My Drive' directory exists
    if os.path.isdir(my_drive_path):
        print("✅ Google Drive is already mounted.")
    else:
        print("🔄 Mounting Google Drive...")
        drive.mount(mount_point)
        # Optional: Verify the mount was successful
        if os.path.isdir(my_drive_path):
            print("✅ Google Drive mounted successfully.")
        else:
            print("❌ Failed to mount Google Drive.")

# Usage
mount_drive_if_not_mounted()

🔄 Mounting Google Drive...
Mounted at /content/drive
✅ Google Drive mounted successfully.


In [3]:
%mkdir /new
%cp -r /content/drive/MyDrive/Data  /new -v



'/content/drive/MyDrive/Data' -> '/new/Data'
'/content/drive/MyDrive/Data/6_1.jpg' -> '/new/Data/6_1.jpg'
'/content/drive/MyDrive/Data/2450_1.jpg' -> '/new/Data/2450_1.jpg'
'/content/drive/MyDrive/Data/2608_1.jpg' -> '/new/Data/2608_1.jpg'
'/content/drive/MyDrive/Data/870_1.jpg' -> '/new/Data/870_1.jpg'
'/content/drive/MyDrive/Data/2680_1.jpg' -> '/new/Data/2680_1.jpg'
'/content/drive/MyDrive/Data/607_1.jpg' -> '/new/Data/607_1.jpg'
'/content/drive/MyDrive/Data/2424_1.jpg' -> '/new/Data/2424_1.jpg'
'/content/drive/MyDrive/Data/2393_1.jpg' -> '/new/Data/2393_1.jpg'
'/content/drive/MyDrive/Data/2749_1.jpg' -> '/new/Data/2749_1.jpg'
'/content/drive/MyDrive/Data/2641_1.jpg' -> '/new/Data/2641_1.jpg'
'/content/drive/MyDrive/Data/2468_1.jpg' -> '/new/Data/2468_1.jpg'
'/content/drive/MyDrive/Data/2721_1.jpg' -> '/new/Data/2721_1.jpg'
'/content/drive/MyDrive/Data/2637_1.jpg' -> '/new/Data/2637_1.jpg'
'/content/drive/MyDrive/Data/2644_1.jpg' -> '/new/Data/2644_1.jpg'
'/content/drive/MyDrive/Dat

In [4]:
import os
import cv2
from ultralytics import YOLO
import numpy as np
from tensorflow.keras.models import load_model
import yaml
import sys
import json
import logging
import argparse
import csv

os.environ["TF_CPP_MIN_LOG_LEVEL"] = (
    "3"  # Suppresses TensorFlow warnings and info messages
)

# Set up logging
logging.basicConfig(
    filename="process.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s",
)


def write_percentage(session_id, content):
    """
    Overwrites the content of the specified file with the given content.

    :param file_path: Path to the file to be overwritten.
    :param content: Content to write into the file.
    """
    with open(session_id, "w") as file:
        file.write(content)


def load_models(yolo_model_path, cow_id_model_path):
    try:
        yolo_model = YOLO(yolo_model_path)
        cow_ID_model = load_model(cow_id_model_path, compile=False)
        return yolo_model, cow_ID_model, None
    except Exception as e:
        logging.error(f"Error loading models: {e}")
        return (
            None,
            None,
            json.dumps(
                {"error": "Failed to load models. Please check the model paths."}
            ),
        )


def load_cow_id_mappings(mapping_file):
    try:
        with open(mapping_file, "r") as f:
            mappings = yaml.safe_load(f)
        return mappings, None
    except Exception as e:
        logging.error(f"Error loading cow ID mappings: {e}")
        return None, json.dumps(
            {"error": "Failed to load cow ID mappings. Please check the mapping file."}
        )


def detect_muzzle(yolo_model, cow_image):
    try:
        results = yolo_model.predict(source=cow_image, verbose=False)
        confidence_scores = results[0].boxes.conf
        max_index = np.argmax(confidence_scores)
        bbox = results[0].boxes.xyxy[max_index]
        bbox = [int(coord) for coord in bbox]
        muzzle_image = cow_image[bbox[1] : bbox[3], bbox[0] : bbox[2]]
        bbox_size = (bbox[3] - bbox[1]) * (bbox[2] - bbox[0])
        max_confidence = confidence_scores[
            max_index
        ]  # Max confidence for the detected muzzle
        return muzzle_image, bbox_size, max_confidence, None
    except Exception as e:
        logging.error(f"Error detecting muzzle: {e}")
        return (
            None,
            None,
            None,
            json.dumps(
                {"error": "Muzzle detection failed. Please check the image and model."}
            ),
        )


def crop_image_center(image, crop_percentage_width=0.20, crop_percentage_height=0.20):
    try:
        height, width, _ = image.shape
        crop_height = int(height * crop_percentage_height)
        crop_width = int(width * crop_percentage_width)
        cropped_image = image[crop_height:-crop_height, crop_width:-crop_width]
        return cropped_image, None
    except Exception as e:
        logging.error(f"Error cropping image: {e}")
        return None, json.dumps(
            {"error": "Image cropping failed. Please check the image dimensions."}
        )


def ensure_target_size(image, target_height=400, target_width=400):
    try:
        height, width, _ = image.shape
        if height < target_height or width < target_width:
            image = cv2.resize(
                image, (target_width, target_height), interpolation=cv2.INTER_LINEAR
            )
        start_x = max(0, (width - target_width) // 2)
        start_y = max(0, (height - target_height) // 2)
        cropped_image = image[
            start_y : start_y + target_height, start_x : start_x + target_width
        ]
        return cropped_image, None
    except Exception as e:
        logging.error(f"Error ensuring target size: {e}")
        return None, json.dumps({"error": "Resizing image to target size failed."})


def prepare_image_for_prediction(image):
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = np.dstack([image] * 3)
        return image, None
    except Exception as e:
        logging.error(f"Error preparing image for prediction: {e}")
        return None, json.dumps({"error": "Image preparation for prediction failed."})


def predict_cow_id(image, model):
    try:
        x1 = np.expand_dims(image, axis=0)
        predictions = model.predict(x1, verbose=0)
        max_confidence = float(np.max(predictions))  # Convert to Python native float
        y = int(np.argmax(predictions, axis=1)[0])  # Convert to Python native int
        return y, max_confidence, None
    except Exception as e:
        logging.error(f"Error predicting cow ID: {e}")
        return (
            None,
            None,
            json.dumps({"error": "Cow ID prediction failed. Please check the image and model."}),
        )


def convert_grayscale_to_color(image):
    try:
        if len(image.shape) == 2:  # Check if the image is single-channel (grayscale)
            color_image = np.dstack([image] * 3)
            return color_image, None
        elif (
            len(image.shape) == 3 and image.shape[2] == 3
        ):  # Check if the image is already in color
            return image, None
        else:
            raise ValueError(
                "Unexpected image format. The image must be either single-channel or 3-channel."
            )
    except Exception as e:
        logging.error(f"Error converting grayscale to color: {e}")
        return None, json.dumps({"error": "Image format conversion failed."})


def determine_quality_from_muzzle(muzzle_image, bbox_size, confidence_score):
    try:
        # Sharpness: Measure the variance of the Laplacian to determine sharpness
        sharpness = cv2.Laplacian(muzzle_image, cv2.CV_64F).var()

        # Logging bbox size, sharpness, and confidence score for debugging
        logging.info(
            f"Bounding Box Size: {bbox_size}, Sharpness: {sharpness}, Confidence Score: {confidence_score}"
        )

        # Adjusted thresholds
        if bbox_size > 25000 and sharpness > 200 and confidence_score > 0.7:
            return "Excellent", None
        elif bbox_size > 20000 and sharpness > 150 and confidence_score > 0.6:
            return "Very Good", None
        elif bbox_size > 15000 and sharpness > 100 and confidence_score > 0.5:
            return "Good", None
        elif bbox_size > 10000 and sharpness > 50 and confidence_score > 0.4:
            return None, json.dumps(
                {"error": "Cow ID prediction failed. Please check the image and model."}
            )
        else:
            return "Poor", json.dumps(
                {"error": "Cow ID prediction failed. Please check the image and model."}
            )
    except Exception as e:
        logging.error(f"Error determining image quality: {e}")
        return "Unknown"

def save_to_csv(file_path, data, fieldnames):
    """
    Save data to a CSV file.

    Parameters:
    - file_path (str): Path to the CSV file.
    - data (dict): Data to write to the CSV file.
    - fieldnames (list): List of column names for the CSV file.
    """
    file_exists = os.path.isfile(file_path)

    with open(file_path, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        # Write the header only if the file does not exist
        if not file_exists:
            writer.writeheader()

        writer.writerow(data)


def main(args, filename):
    error_occurred = None
    try:
        write_percentage(args.session, "Completed 5%. Loading Models ...")
        yolo_model, cow_ID_model, error = load_models(
            args.yolo_model, args.cow_id_model
        )
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 10%. Loading Cow's Mapping Database...")
        int_to_ID_161, error = load_cow_id_mappings(args.mapping_file)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 25%. Uploading Image to Model...")
        cow_image = cv2.imread(args.image)
        if cow_image is None:
            error_occurred = json.dumps({"error": "Image not found or unable to load."})
            sys.stderr.write(error_occurred)
            return

        write_percentage(args.session, "Completed 50%. Detecting Muzzle...")
        muzzle_image, bbox_size, confidence_score, error = detect_muzzle(
            yolo_model, cow_image
        )
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 55%. Cropping the Muzzle area...")
        cropped_muzzle_image, error = crop_image_center(muzzle_image)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 70%. Converting the GrayScale image to Color ...")
        color_muzzle_image, error = convert_grayscale_to_color(cropped_muzzle_image)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 74%. Preparing Image for Prediction ...")
        prepared_image, error = prepare_image_for_prediction(color_muzzle_image)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 76%. Cross-checking Target Image Size ...")
        image_front, error = ensure_target_size(prepared_image)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, "Completed 90%. Predicting Cow ID ...")
        predicted_label, confidence, error = predict_cow_id(image_front, cow_ID_model)
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        # Determine quality based on muzzle detection
        write_percentage(args.session, "Completed 95%. Determining Model's Quality from Muzzle ...")
        quality, error = determine_quality_from_muzzle(
            muzzle_image, bbox_size, confidence_score
        )
        if error:
            sys.stderr.write(error)
            error_occurred = error
            return

        write_percentage(args.session, f"Completed 95%. Model's Quality of Muzzle: {quality} ...")
        # Now, use the color image for further processing if needed
        # For demonstration, let's save the color image to a file
        write_percentage(args.session, "Completed 99%. Saving Image to Disk ...")
        cv2.imwrite("color_file.png", color_muzzle_image)

        data = {
            "image_path": filename,
            "cow_id": int_to_ID_161[predicted_label],
            "confidence_score": round(confidence, 2),
            "temperature": args.temperature,
            "threshold": args.threshold,
            "quality": quality,
            "error": None,
        }
        write_percentage(args.session, "Completed 100%. Completed Cow Identification, processing results ...")
        print(json.dumps(data))

        fieldnames = list(data.keys())
        save_to_csv('report.csv', data, fieldnames)
    except Exception as e:
        error_json = json.dumps({"error": str(e)})
        sys.stderr.write(error_json)
        logging.error(f"Error in main function: {e}")
        error_occurred = error_json
    finally:
        if error_occurred:
          data = {
            "image_path": filename,
            "cow_id":'',
            "confidence_score": '',
            "temperature": '',
            "threshold": '',
            "quality":'',
            "error": error_occurred,
          }
          fieldnames = list(data.keys())
          save_to_csv('report.csv', data, fieldnames)


if __name__ == "__main__":
    # Define the directory containing images and other files
    directory = "/new/Data"
    print(directory)
    # Supported image extensions
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif')

    all_files = os.listdir(directory)
    image_files = [f for f in all_files if f.lower().endswith(image_extensions)]
    total_files = len(image_files)

    processed_files = 0
    # Loop through the directory to find all image files
    for filename in os.listdir(directory):
        if filename.lower().endswith(image_extensions):
            # Create the full path to the image
            image_path = os.path.join(directory, filename)
            processed_files += 1


            # Create the args Namespace with the current image
            args = argparse.Namespace(
                image=image_path,
                yolo_model=os.path.join(directory, "best.pt"),        # Path to the YOLO model
                cow_id_model=os.path.join(directory, "ResNet50_cow_id_all.h5"),  # Path to the cow ID model
                mapping_file=os.path.join(directory, "int_to_ID_all_cows.yaml"), # Path to the YAML mapping file
                temperature=0.5,                                      # Adjust temperature as needed
                threshold=0.5,                                        # Adjust threshold as needed
                session=filename+'.txt'                                      # Session string
            )

            # Call the main function with the current args
            print(f"Processing file {processed_files} of {total_files} -  {filename}")
            main(args,filename)





/new/Data
Processing file 1 of 349 -  3026_1.jpg
{"image_path": "3026_1.jpg", "cow_id": "3026", "confidence_score": 1.0, "temperature": 0.5, "threshold": 0.5, "quality": "Excellent", "error": null}
Processing file 2 of 349 -  2261_1.jpg
{"image_path": "2261_1.jpg", "cow_id": "2856", "confidence_score": 0.75, "temperature": 0.5, "threshold": 0.5, "quality": "Excellent", "error": null}
Processing file 3 of 349 -  2641_1.jpg


{"error": "Cow ID prediction failed. Please check the image and model."}

Processing file 4 of 349 -  2528_1.jpg
{"image_path": "2528_1.jpg", "cow_id": "2528", "confidence_score": 1.0, "temperature": 0.5, "threshold": 0.5, "quality": "Very Good", "error": null}
Processing file 5 of 349 -  2750_1.jpg


{"error": "Cow ID prediction failed. Please check the image and model."}

Processing file 6 of 349 -  2745_1.jpg


...

Processing file 344 of 349 -  2915_1.jpg
{"image_path": "2915_1.jpg", "cow_id": "2915", "confidence_score": 1.0, "temperature": 0.5, "threshold": 0.5, "quality": "Good", "error": null}
Processing file 345 of 349 -  2863_1.jpg
{"image_path": "2863_1.jpg", "cow_id": "2863", "confidence_score": 1.0, "temperature": 0.5, "threshold": 0.5, "quality": "Good", "error": null}
Processing file 346 of 349 -  2791_1.jpg


{"error": "Cow ID prediction failed. Please check the image and model."}

Processing file 347 of 349 -  2513_1.jpg
{"image_path": "2513_1.jpg", "cow_id": "2513", "confidence_score": 1.0, "temperature": 0.5, "threshold": 0.5, "quality": "Very Good", "error": null}
Processing file 348 of 349 -  2942_1.jpg


{"error": "Cow ID prediction failed. Please check the image and model."}

Processing file 349 of 349 -  3036_1.jpg
{"image_path": "3036_1.jpg", "cow_id": "3036", "confidence_score": 0.77, "temperature": 0.5, "threshold": 0.5, "quality": "Excellent", "error": null}


In [38]:
import pandas as pd
import re

def extract_integer_id(filename):
    """
    Extract the first integer ID from the filename.

    Parameters:
    filename (str): The name of the file to extract the ID from.

    Returns:
    int or str: The extracted integer ID, or an empty string if no integer ID is found.
    """
    # Extract the first sequence of digits from the filename before any underscore
    match = re.match(r'(\d+)', filename.split('_')[0])
    if match:
        return int(match.group(1))
    return ''

# Load the CSV file
csv_file_path = 'report.csv'  # Update the path as needed
df = pd.read_csv(csv_file_path)

# Initialize counters and results list
passed_count = 0
failed_count = 0
error_count = 0
results = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    # Extract necessary information
    image_path = row['image_path']
    cow_id = row['cow_id']
    actual_cow_id = extract_integer_id(image_path)
    error_message = 'false'

    # Handle errors
    if not pd.isnull(row['error']):  # Check if 'error' column is not NaN
        status = 'error'
        error_count += 1
        error_message = row['error']
        results.append({
          'Actual COW ID': actual_cow_id,
          'Cow ID': cow_id if not pd.isna(cow_id) else 'N/A',
          'Status': status,
          'Error': error_message
        })
        continue


    if pd.isna(cow_id):  # No cow_id provided
        status = 'failed'
        failed_count += 1
    else:
        cow_id = int(cow_id)
        # Compare actual cow ID with the provided cow ID
        if actual_cow_id == cow_id:
            status = 'passed'
            passed_count += 1
        else:
            status = 'failed'
            failed_count += 1

    results.append({
        'Actual COW ID': actual_cow_id,
        'Cow ID': cow_id if not pd.isna(cow_id) else 'N/A',
        'Status': status,
        'Error': error_message
    })

# Create a DataFrame for the results table
results_df = pd.DataFrame(results)

# Print the counts
print(f'Passed: {passed_count}')
print(f'Failed: {failed_count}')
print(f'Error: {error_count}')

# Print the results table
import IPython.display as display
display.display(results_df)


Passed: 209
Failed: 47
Error: 93


,Actual COW ID,Cow ID,Status,Error
0,3026,3026,passed,false
1,2261,2856,failed,false
2,2641,N/A,error,"{""error"": ""Cow ID prediction failed. Please ch..."
3,2528,2528,passed,false
4,2750,N/A,error,"{""error"": ""Cow ID prediction failed. Please ch..."
...,...,...,...,...
344,2863,2863,passed,false
345,2791,N/A,error,"{""error"": ""Cow ID prediction failed. Please ch..."
346,2513,2513,passed,false
347,2942,N/A,error,"{""error"": ""Cow ID prediction failed. Please ch..."
